In [ ]:
cd assign_529

In [ ]:
import csv
moviemap={}
moviemap_inv={}
with open('./dataset/ratings.csv','rt')as f:
    data = csv.reader(f)
    i=0
    cnt=0
    for row in data:
        if i!=0:
            x=int(row[1])
            if x in moviemap.keys():
                print("present")
            else:
                moviemap[int(row[1])]=cnt
                moviemap_inv[cnt]=int(row[1])
                cnt=cnt+1
        i=i+1
#print(cnt) cnt = 9724
rows=611
cols=9725
rating_mat=[[0 for i in range(cols)] for j in range(rows)]
test_mat=[[0 for i in range(cols)] for j in range(rows)]
with open('./dataset/ratings.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        if i!=0:
            if i%5==0:
                test_mat[int(row[0])][moviemap[int(row[1])]]=float(row[2])
            else:
                rating_mat[int(row[0])][moviemap[int(row[1])]]=float(row[2])
        i=i+1

# User-User Based Implementations

sim_u is the matrix for user user based pearson similarity

In [9]:
sim_u=[[0 for i in range(rows)] for j in range(rows)]#similarity matrix using pearson similarity (user-user)

pearson function calculates pearson similarity between two users u1 and u2 on the basis of matrix r:

In [6]:
import math
def pearson(r,u1, u2):
    s1=0
    n1=0
    s2=0
    n2=0
    for i in range(cols):
        s1 = s1+r[u1][i]
        if r[u1][i]!=0:
            n1 = n1+1
        s2 = s2+r[u2][i]
        if r[u2][i]!=0:
            n2 = n2+1
    if n1==0:
        s1=0
    else:
        s1 = s1/n1
    if n2==0:
        s2=0
    else:
        s2 = s2/n2
    num=0
    for i in range(cols):
        if r[u1][i]!=0 and r[u2][i]!=0:
            num = num+((r[u1][i]-s1)*(r[u2][i]-s2))
    den1=0
    den2=0
    for i in range(cols):
        if r[u1][i]!=0:
            den1 = den1+((r[u1][i]-s1)*(r[u1][i]-s1))
        if r[u2][i]!=0:
            den2 = den2+((r[u2][i]-s2)*(r[u2][i]-s2))
    if den1==0 and den2==0:
        return 1
    else:
        if den1==0 or den2==0:
            return 0
    den1 = math.sqrt(den1)
    den2 = math.sqrt(den2)
#     print("num: "+str(num)+" den: "+str(den1)+" "+str(den2))
    ans = num/(den1*den2)
    return ans

In [7]:
print(pearson(rating_mat,20,30))

0.014064840478826626


For calculating similarity matrix and writing the values in a file:

In [ ]:
#execute only if want to make similarity matrix of user user again
# f = open("sim.csv","w+")
# for i in range(rows):
#     j=i+1
#     while j!=rows:
#         sim_u[i][j]=pearson(rating_mat,i,j)
#         print("i: "+str(i)+" j: "+str(j)+" "+str(sim_u[i][j]))
#         f.write(str(i)+", "+str(j)+", "+str(sim_u[i][j])+"\n")
#         j=j+1

Reading similarity matrix from file:

In [10]:
with open('./sim.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        sim_u[int(row[0])][int(row[1])]=float(row[2])
        sim_u[int(row[1])][int(row[0])]=float(row[2])    

Finding top k most similar users:

In [8]:
def find_top_k(u, smat, k):
    lis =[]
    for i in range(rows):
        if i!=u:
#             print("len: "+str(len(smat))+" "+str(len(smat[0])))
#             print("u: "+str(u)+" i: "+str(i))
            lis.append((i,smat[u][i]))
    lis.sort(key = lambda x: x[1]) 
    lis1=[]
    cnt=0
    for t in lis:
        lis1.append(t[0])
        cnt=cnt+1
        if cnt==k:
            break
    return lis1

Function to predict the rating of user u for item j using the k most similar users:

In [11]:
def predict(u,j,smat,rmat,k):
    lis = find_top_k(u,smat,k)
#     print(lis)
    num=0
    den=0
    for i in range(k):
        if(rmat[lis[i]][j]!=0):
#             print("hey: "+str(lis[i]))
            den=den+smat[u][lis[i]]
            num = num+smat[u][lis[i]]*rmat[lis[i]][j]
    if den==0:
        return 0
    return (num/den)

Function for finding the mean absolute error:

In [61]:
def test_pred(tmat,smat,k):
    s=0
    cnt=0
    for i in range(rows):
        for j in range(cols):
            if tmat[i][j]!=0:
                x = predict(i,j,smat,tmat,k)
                y = tmat[i][j]
                s = s+abs(x-y)
                cnt = cnt+1
#                 if cnt%100==0:
#                     print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
    if cnt==0:
        return -1
    else:
        return (s/cnt)

Function for finding the rms error:

In [12]:
def test_pred_rms(tmat,smat,k):
    s=0
    cnt=0
    for i in range(rows):
        for j in range(cols):
            if tmat[i][j]!=0:
                x = predict(i,j,smat,tmat,k)
                y = tmat[i][j]
                s = s+(x-y)*(x-y)
                cnt = cnt+1
#                 if cnt%100==0:
#                     print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
    if cnt==0:
        return -1
    else:
        return math.sqrt(s/cnt)

Function for finding cosine similarity between users u1 and u2:

In [91]:
def cosine(r,u1,u2):
    num=0
    for i in range(cols):
        if r[u1][i]!=0 and r[u2][i]!=0:
            num = num+(r[u1][i]*r[u2][i])
    den1=0
    den2=0
    for i in range(cols):
        den1 = den1+(r[u1][i]*r[u1][i])
        den2 = den2+(r[u2][i]*r[u2][i])
    den1 = math.sqrt(den1)
    den2 = math.sqrt(den2)
    if den1==0 and den2==0:
        return 1
    else:
        if den1==0 or den2==0:
            return 0
    return num/(den1*den2)

In [71]:
sim_u_c=[[0 for i in range(rows)] for j in range(rows)]#similarity matrix using cosine similarity (user-user)

In [72]:
with open('./sim_u_c.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        sim_u_c[int(row[0])][int(row[1])]=float(row[2])
        sim_u_c[int(row[1])][int(row[0])]=float(row[2])    

In [ ]:
#execute only if wish to construct cosine based user user matrix again
# f.close()
# f = open("sim_u_c.csv","w+")
# for i in range(rows):
#     j=i+1
#     while j!=rows:
#         x=pearson(rating_mat,i,j)
#         sim_u_c[i][j]=x
#         sim_u_c[j][i]=x
# #         if i%50==0:
#         f.write(str(i)+", "+str(j)+", "+str(sim_u_c[i][j])+"\n")
#         print("i: "+str(i)+" j: "+str(j)+" "+str(sim_u_c[i][j]))
#         j=j+1
# f.close()

In [73]:
print(test_pred_rms(test_mat,sim_u_c,20))

3.2730325168250123


# Mean Centered Matrix (User based)

Calculation of mean centered matrix:

In [17]:
mean_centered=[[0 for i in range(cols)] for j in range(rows)]

In [18]:
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s = s+rating_mat[i][j]
        if rating_mat[i][j]!=0:
            cnt=cnt+1
    if cnt==0:
        s=0
    else:
        s=s/cnt
    for j in range(cols):
        if rating_mat[i][j]!=0:
            mean_centered[i][j]=rating_mat[i][j]-s

Prediction function for mean centered matrix:

In [21]:
def predict_mean_centered(u,j,smat,rmat,mean_centered,k):
    lis = find_top_k(u,smat,k)
#     print(lis)
    num=0
    den=0
    avg=0
    cnt=0
    for i in range(cols):
#         print("cols: "+str(cols))
#         print("u: "+str(u)+" i: "+str(i))
        avg=avg+rmat[u][i]
        if rmat[u][i]!=0:
            cnt=cnt+1
    if cnt==0:
        avg=0
    else:
        avg=avg/cnt
    for i in range(k):
        if(rmat[lis[i]][j]!=0):
#             print("hey: "+str(lis[i]))
            den=den+smat[u][lis[i]]
            num = num+smat[u][lis[i]]*mean_centered[lis[i]][j]
    if den==0:
        return 0
    return (avg+(num/den))

Function for calulating mean absolute error for mean centered matrix:

In [8]:
def test_pred_mean_centered(tmat,smat,rmat,mc_mat,k):
    s=0
    cnt=0
    for i in range(rows):
        for j in range(cols):
            if tmat[i][j]!=0:
                x = predict_mean_centered(i,j,smat,rmat,mc_mat,k)
                y = tmat[i][j]
                s = s+abs(x-y)
                cnt = cnt+1
#                 if cnt%100==0:
                print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
    return (s/cnt)

Function for calulating rms error for mean centered matrix:

In [16]:
def test_pred_mean_centered_rms(tmat,smat,rmat,mc_mat,k):
    s=0
    cnt=0
    for i in range(rows):
        for j in range(cols):
            if tmat[i][j]!=0:
                x = predict_mean_centered(i,j,smat,rmat,mc_mat,k)
                y = tmat[i][j]
                s = s+(x-y)*(x-y)
                cnt = cnt+1
#                 if cnt%100==0:
#                 print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
    return math.sqrt(s/cnt)

In [80]:
print(test_pred_mean_centered_rms(test_mat,sim_u,rating_mat,mean_centered,200))#1.9943681371727275

23.10056788567136


In [ ]:
print(test_pred_mean_centered(test_mat,sim_u_c,rating_mat,mean_centered,200))#1.7925364356314288

# Normalized Matrix (User Based)

In [26]:
normalized=[[0 for i in range(cols)] for j in range(rows)]

In [ ]:
normalized=[[0 for i in range(cols)] for j in range(rows)]
for i in range(rows):
    s=0
    cnt=0
    for j in range(cols):
        s = s+rating_mat[i][j]
        if rating_mat[i][j]!=0:
            cnt=cnt+1
    if cnt==0:
        s=0
    else:
        s=s/cnt
    print("s: "+str(s))
    dev=0
    for j in range(cols):
        if rating_mat[i][j]!=0:
            dev = dev+(rating_mat[i][j]-s)*(rating_mat[i][j]-s)
    if cnt==0:
        dev=0
    else:
        dev = dev/cnt
    dev = math.sqrt(dev)
    for j in range(cols):
        if dev!=0:
            normalized[i][j]=(abs(rating_mat[i][j]-s))/dev
# print("cnt: "+str(cnt))
#             normalized[j][i]=(rating_mat[i][j]-s)/dev
#     print("i: "+str(i)+" j: "+str(j))

Prediction function for normalized matrix:

In [28]:
def predict_normalized(u,j,smat,rmat,normal_mat,k):
    lis = find_top_k(u,smat,k)
    print(lis)
    num=0
    den=0
    avg=0
    cnt=0
    for i in range(cols):
        avg=avg+normal_mat[u][i]
        if normal_mat[u][i]!=0:
#             print("hello")
            cnt=cnt+1
    if cnt==0:
        avg=0
    else:
        avg=avg/cnt
    dev=0
#     print("cnt: "+str(cnt)+" "+str(avg))
    for i in range(cols):
        if normal_mat[u][i]!=0:
            dev=dev+(normal_mat[u][i]-avg)*(normal_mat[u][i]-avg)
    if cnt==0:
        dev=0
    else:
        dev = math.sqrt(dev/cnt)
#     print("dev: "+str(dev))
    for i in range(k):
#         print("rmat: "+str(rmat[lis[i]][j]))
        if(rmat[lis[i]][j]!=0):
#             print("hey: "+str(lis[i]))
            den=den+smat[u][lis[i]]
#             print(smat[u][lis[i]])
            num = num+smat[u][lis[i]]*normal_mat[lis[i]][j]
    if den==0:
        return 0
    print("avg: "+str(avg)+" dev: "+str(dev))
    return (avg+(dev*num/den))

In [29]:
predict_normalized(20,30,sim_u,rating_mat,normalized,50)

[39, 401, 194, 34, 566, 405, 339, 336, 377, 155, 272, 67, 416, 134, 497, 123, 120, 103, 238, 419, 130, 81, 125, 491, 412, 538, 37, 390, 369, 458, 459, 598, 572, 3, 41, 247, 285, 551, 59, 176, 316, 60, 148, 76, 394, 313, 462, 79, 283, 562]
avg: 3.0216156694202985 dev: 0.3309087467851954


3.2176620791532535

Finding the mean absolute error for normalized matrix:

In [30]:
def test_pred_normalized(tmat,smat,rmat,nmat,k):
    s=0
    cnt=0
    for i in range(rows):
        for j in range(cols):
            if tmat[i][j]!=0:
                x = predict_normalized(i,j,smat,rmat,nmat,k)
                y = tmat[i][j]
                s = s+abs(x-y)
                cnt = cnt+1
#                 if cnt%100==0:
#                 print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
    return (s/cnt)

Finding the rms error for normalized matrix:

In [84]:
def test_pred_normalized_rms(tmat,smat,rmat,nmat,k):
    s=0
    cnt=0
    for i in range(rows):
        for j in range(cols):
            if tmat[i][j]!=0:
                x = predict_normalized(i,j,smat,rmat,nmat,k)
                y = tmat[i][j]
                s = s+(x-y)*(x-y)
                cnt = cnt+1
#                 if cnt%100==0:
#                 print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
    return math.sqrt(s/cnt)

In [ ]:
print(test_pred_normalized(test_mat,sim_u,rating_mat,normalized,200))#11.362225790470779

In [ ]:
print(test_pred_normalized(test_mat,sim_u,rating_mat,normalized,50))#11.362225790470779

In [ ]:
print(test_pred_normalized(test_mat,sim_u_c,rating_mat,normalized,300))

In [ ]:
print(test_pred_normalized_rms(test_mat,sim_u_c,rating_mat,normalized,200))

# Countering Long Tail Effect

Inverse User Frequency is calculated for each user in iuf:

In [13]:
#taking measures to counter long tail effect
iuf = [0 for i in range(cols)]
for j in range(cols):
    cnt=0
    for i in range(rows):
        if rating_mat[i][j]!=0:
            cnt=cnt+1
    iuf[j]=cnt/rows

Weighted similarity is calculated for users u1 and u2 using the iuf weights

In [130]:
def similarity_iuf(r,u1,u2,arr_iuf):
    s1=0
    n1=0
    s2=0
    n2=0
    for i in range(cols):
        s1 = s1+r[u1][i]
        if r[u1][i]!=0:
            n1 = n1+1
        s2 = s2+r[u2][i]
        if r[u2][i]!=0:
            n2 = n2+1
    if n1==0:
        s1=0
    else:
        s1 = s1/n1
    if n2==0:
        s2=0
    else:
        s2 = s2/n2
    num=0
    for i in range(cols):
        if r[u1][i]!=0 and r[u2][i]!=0:
            num = num+(arr_iuf[i]*(r[u1][i]-s1)*(r[u2][i]-s2))
    den1=0
    den2=0
    for i in range(cols):
        if r[u1][i]!=0:
            den1 = den1+(arr_iuf[i]*(r[u1][i]-s1)*(r[u1][i]-s1))
        if r[u2][i]!=0:
            den2 = den2+(arr_iuf[i]*(r[u2][i]-s2)*(r[u2][i]-s2))
    if den1==0 and den2==0:
        return 1
    else:
        if den1==0 or den2==0:
            return 0
    den1 = math.sqrt(den1)
    den2 = math.sqrt(den2)
#     print("num: "+str(num)+" den: "+str(den1)+" "+str(den2))
    ans = num/(den1*den2)
    return ans

In [14]:
sim_u_iuf=[[0 for i in range(rows)] for j in range(rows)]

In [ ]:
# execute only if wish to compute similarity matrix again
# f = open("sim_u_iuf.csv","w+")
# for i in range(rows):
#     j=i+1
#     while j!=rows:
#         sim_u_iuf[i][j]=similarity_iuf(rating_mat,i,j,iuf)
#         if j%100==0:
#             print("i: "+str(i)+" j: "+str(j)+" "+str(sim_u_iuf[i][j]))
#         f.write(str(i)+", "+str(j)+", "+str(sim_u_iuf[i][j])+"\n")
#         j=j+1
# f.close()

In [19]:
with open('./sim_u_iuf.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        sim_u_iuf[int(row[0])][int(row[1])]=float(row[2])
        sim_u_iuf[int(row[1])][int(row[0])]=float(row[2])  

In [25]:
print(test_pred_mean_centered_rms(test_mat,sim_u_iuf,rating_mat,mean_centered,100))

1.9734430622146786


# Item-Item Based Ratings (Mean-Centered)

Finding the transpose of required matrices:

In [96]:
#item-item
temp=rows
rows=cols
cols=temp
rating_mat_trans = [[rating_mat[j][i] for j in range(len(rating_mat))] for i in range(len(rating_mat[0]))] 
test_mat_trans = [[test_mat[j][i] for j in range(len(test_mat))] for i in range(len(test_mat[0]))] 

In [97]:
#mean_centering
mean_centered_t=[[0 for i in range(cols)] for j in range(rows)]
for i in range(rows):
    avg=0
    cnt=0
    for j in range(cols):
        avg = avg+rating_mat_trans[i][j]
        if rating_mat_trans[i][j]!=0:
            cnt=cnt+1
    if cnt==0:
        avg=0
    else:
        avg=avg/cnt
    for j in range(cols):
        if rating_mat_trans[i][j]!=0:
            mean_centered_t[i][j]=rating_mat_trans[i][j]-avg

Finding the similarity between two items using pearson based similarity in sim_i matrix

In [98]:
sim_i=[[0 for i in range(rows)] for j in range(rows)]

In [ ]:
# execute only if wish to construct cosine based item item matrix again
# f = open("sim_i.csv","w+")
# for i in range(rows):
#     j=i+1
#     while j!=rows:
#         sim_i[i][j]=pearson(rating_mat_trans,i,j)
#         if j%100==0:
#             print("i: "+str(i)+" j: "+str(j)+" "+str(sim_i[i][j]))
#         f.write(str(i)+", "+str(j)+", "+str(sim_i[i][j])+"\n")
#         j=j+1
# f.close()

In [ ]:
with open('./sim_i.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        if(len(row)>=2):
            if int(row[1])%500==0 and int(row[0])%100==0:
                print(row[0]+" "+row[1]+" "+row[2])
            sim_i[int(row[0])][int(row[1])]=float(row[2])
            sim_i[int(row[1])][int(row[0])]=float(row[2])    

In [ ]:
# f.close()
# f = open("sim_i.csv","a+")
# f.write("\n")
# for i in range(3074,3075):
#     j=6168
#     while(j!=rows):
#         sim_i[i][j]=pearson(rating_mat_trans,i,j)
#         if j%100==0:
#             print("i: "+str(i)+" j: "+str(j)+" "+str(sim_i[i][j]))
#         f.write(str(i)+", "+str(j)+", "+str(sim_i[i][j])+"\n")
#         j=j+1
# for i in range(3075,rows):
#     j=i+1
#     while j!=rows:
#         sim_i[i][j]=pearson(rating_mat_trans,i,j)
#         if j%100==0:
#             print("i: "+str(i)+" j: "+str(j)+" "+str(sim_i[i][j]))
#         f.write(str(i)+", "+str(j)+", "+str(sim_i[i][j])+"\n")
#         j=j+1
# f.close()

Finding the similarity between two items using cosine based similarity in sim_i matrix

In [ ]:
sim_i_c=[[0 for i in range(rows)] for j in range(rows)]
with open('./sim_i_c.csv','rt')as f:
    data = csv.reader(f)
    i=0
    for row in data:
        if(len(row)>=2):
            if int(row[1])%500==0 and int(row[0])%100==0:
                print(row[0]+" "+row[1]+" "+row[2])
            sim_i_c[int(row[0])][int(row[1])]=float(row[2])
            sim_i_c[int(row[1])][int(row[0])]=float(row[2])    

In [101]:
print(test_pred_mean_centered_rms(test_mat_trans,sim_i,rating_mat_trans,mean_centered_t,1500))#test_pred_mean_centered(tmat,smat,rmat,mc_mat,k) 0.893

1.3007271507586615


In [21]:
sim_i_c=[[0 for i in range(rows)] for j in range(rows)]

In [ ]:
# f = open("sim_i_c.csv","w+")
# for i in range(rows):
#     j=i+1
#     while j!=rows:
#         sim_i_c[i][j]=cosine(rating_mat_trans,i,j)
#         if j%100==0:
#             print("i: "+str(i)+" j: "+str(j)+" "+str(sim_i_c[i][j]))
#         f.write(str(i)+", "+str(j)+", "+str(sim_i_c[i][j])+"\n")
#         j=j+1
# f.close()

# Approach for finding the best possible ratings using Item-Item and User-User

In [54]:
#combining item-item and user-user approach
s=0
cnt=0
rows_new =rows
cols_new =cols
for i in range(rows_new):
    for j in range(cols_new):
        if test_mat_trans[i][j]!=0:
            x1 = predict_mean_centered(i,j,sim_i,rating_mat_trans,mean_centered_t,1800)
            rows=cols_new
            cols=rows_new
            x2 = predict_mean_centered(j,i,sim_u,rating_mat,mean_centered,100)
            rows=rows_new
            cols=cols_new
            x=(x1+x2)/2
            y = test_mat_trans[i][j]
            s = s+abs(x-y)
            cnt = cnt+1
#                 if cnt%100==0:
            if i%500==0 and j%100==0:
                print("i: "+str(i)+" j: "+str(j)+" x: "+str(x)+" y: "+str(y))
print (s/cnt)

i: 0 j: 200 x: 4.282365028957527 y: 3.5
i: 1000 j: 500 x: 3.4985725937906405 y: 3.0
i: 4500 j: 200 x: 1.5005139142631048 y: 3.0
1.1090616199417764
